# Data Challenge: H-index Prediction

Michael Fotso Fotso, Tristan François and Christian Kotait

## Data Generation

### First preprocessing

Create a file `processed_data.csv` in the folde `../tmp/` with a few features related to the graph and two features related to a **fasttext model**.

It took about 30 minutes on our machines.

In [ ]:
from preprocess_utils import store_full_dataset_with_features

store_full_dataset_with_features(from_scratch=True, vectorize=True)

### Add a closeness centrality feature

It took about 2 hours on our machines.

In [ ]:
from utils import get_closeness
from preprocess_utils import PROCESSED_DATA_PATH, add_features, get_processed_data

data = get_processed_data(split=False)
closeness = get_closeness()
data = add_features(data, closeness)
data.to_csv(PROCESSED_DATA_PATH)

### Add doc2vec features

Add 20 features from a doc2vec model.

It took about 5 hours on our machines.

In [ ]:
from d2vec import add_do2vec_to_whole_dataset
from preprocess_utils import PROCESSED_DATA_PATH, get_processed_data

data = get_processed_data(split=False)
data = add_do2vec_to_whole_dataset(data)
data.to_csv(PROCESSED_DATA_PATH)

### Add tf-idf features

Add `n_features` from a tf-idf vectorizer.

It took a few minutes on our machine but it requires a lot of ram.

In [ ]:
from preprocess_utils import PROCESSED_DATA_PATH, get_processed_data, add_tf_idf

data = get_processed_data(split=False)
data = add_tf_idf(data, n_features=1000)
data.to_csv(PROCESSED_DATA_PATH)

## Model training

In [ ]:
iterations = 1000
task_type = "GPU"

In [ ]:
from catboost import CatBoostRegressor
from preprocess_utils import get_submission_data
from read_data import get_test_data

X_train, y_train, X_test, y_test = get_submission_data()

model_cat = CatBoostRegressor(
    random_state=1,
    iterations=iterations,
    task_type=task_type,
    depth=8
)

model_cat.fit(X_train, y_train)

y_pred = model_cat.predict(X_test)

test, _ = get_test_data()
test["hindex"] = y_pred
submission = test[["author", "hindex"]]
submission.to_csv("../tmp/submission.csv", index=None)